### Capítulo 14: Redes Neurais Recorrentes (RNN)

### Resumo:

Este capítulo introduz as **Redes Neurais Recorrentes (RNNs)**, uma classe de redes neurais que se destaca na análise de **dados sequenciais** e na **previsão de eventos futuros**. Ao contrário das redes neurais *feedforward* tradicionais, as RNNs podem trabalhar com sequências de comprimento arbitrário, tornando-as extremamente úteis para tarefas como:
*   Análise de **séries temporais** (por exemplo, preços de ações).
*   **Processamento de Linguagem Natural (PNL)**, incluindo tradução automática, fala para texto e análise de sentimentos.
*   Sistemas de **condução autônoma** (previsão de trajetórias de carros).

### Neurônios Recorrentes

Um neurônio recorrente é semelhante a um neurônio em uma rede *feedforward*, mas com uma **conexão apontando para trás**, alimentando sua própria saída de volta para si mesmo no próximo passo de tempo. Isso permite que a rede mantenha uma **memória** dos passos de tempo anteriores.

*   **Desenrolamento no Tempo**: Para entender como funciona, a rede é conceptualmente "desenrolada" ao longo do eixo do tempo, criando uma cópia do neurônio para cada passo de tempo. Cada cópia recebe as entradas do passo de tempo atual e a saída da cópia anterior.
*   **Tipos de RNNs**: Dependendo das entradas e saídas, as RNNs podem ser classificadas em:
    *   **Sequência para Sequência**: A rede recebe uma sequência de entradas e produz uma sequência de saídas (por exemplo, tradução automática palavra por palavra).
    *   **Sequência para Vetor**: A rede recebe uma sequência de entradas e produz um vetor de saída único (por exemplo, classificação de sentimento de uma frase).
    *   **Vetor para Sequência**: A rede recebe um vetor de entrada único e produz uma sequência de saídas (por exemplo, geração de uma legenda para uma imagem).
    *   **Sequência para Sequência Atrasada**: Um tipo onde o *encoder* lê a sequência completa antes que o *decoder* comece a produzir a saída (comum em tradução automática para obter melhor contexto).

### RNNs Básicas no TensorFlow

O TensorFlow oferece funções para criar RNNs de forma eficiente:

*   **Desenrolamento Estático (`tf.contrib.rnn.static_rnn()`)**: Constrói um grafo que contém uma célula para cada passo de tempo. Embora fácil de entender, pode resultar em grafos muito grandes para sequências longas, levando a erros de falta de memória (OOM) durante a retropropagação.
    ```python
    import tensorflow as tf
    from tensorflow.contrib.rnn import BasicRNNCell, static_rnn

    # Exemplo simplificado
    n_inputs = 3
    n_neurons = 5
    n_steps = 2
    X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
    X_seqs = tf.unstack(tf.transpose(X, perm=))

    basic_cell = BasicRNNCell(num_units=n_neurons)
    output_seqs, states = static_rnn(basic_cell, X_seqs, dtype=tf.float32)
    outputs = tf.transpose(tf.stack(output_seqs), perm=)
    ```
*   **Desenrolamento Dinâmico (`tf.nn.dynamic_rnn()`)**: Utiliza uma operação `while_loop()` para rodar a célula o número apropriado de vezes. Isso resulta em um **grafo muito menor e mais legível** no TensorBoard, e permite **trocar memória da GPU para a CPU** durante a retropropagação (`swap_memory=True`) para evitar erros de OOM.
    ```python
    import tensorflow as tf
    from tensorflow.contrib.rnn import BasicRNNCell

    # Exemplo simplificado
    n_inputs = 3
    n_neurons = 5
    n_steps = 2
    X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])

    basic_cell = BasicRNNCell(num_units=n_neurons)
    outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)
    ```
*   **Sequências de Entrada de Comprimento Variável**: Para lidar com sequências de diferentes tamanhos, o parâmetro `sequence_length` pode ser usado em `dynamic_rnn()` (ou `static_rnn()`) para indicar o comprimento de cada sequência na entrada. As saídas para passos de tempo além do comprimento real da sequência serão zero.
*   **Sequências de Saída de Comprimento Variável**: Se o comprimento da sequência de saída não for conhecido antecipadamente, uma solução comum é definir um **token de fim de sequência (EOS token)**. Qualquer saída após o EOS deve ser ignorada na função de custo.
*   **Classificação de Sequências (Exemplo MNIST)**: As RNNs podem ser usadas para classificação, onde a saída do último passo de tempo é alimentada a uma camada *fully connected* com uma função de ativação softmax para prever a classe.
*   **Previsão de Séries Temporais**: Para prever o próximo valor em uma série temporal, as RNNs geralmente têm uma camada de projeção de saída (`OutputProjectionWrapper`) que aplica uma camada *fully connected* linear sobre cada saída do passo de tempo.

### RNNs Profundas

É comum **empilhar várias camadas de células** para criar **RNNs profundas**. No TensorFlow, isso é feito usando `tf.contrib.rnn.MultiRNNCell`.

*   **Distribuição entre Múltiplas GPUs**: Para distribuir RNNs profundas de forma eficiente em várias GPUs (conforme discutido no Capítulo 12), cada camada pode ser fixada a uma GPU diferente usando um *wrapper* de célula personalizado (`DeviceCellWrapper`).
*   **Aplicação de Dropout**: Para evitar *overfitting* em RNNs profundas, o **dropout** (introduzido no Capítulo 11) pode ser aplicado. Para aplicar dropout entre as camadas da RNN, é necessário usar `tf.contrib.rnn.DropoutWrapper`, configurando `input_keep_prob` ou `output_keep_prob`. É crucial lembrar que o dropout deve ser aplicado **apenas durante o treinamento**, o que pode exigir o uso de um *placeholder* `is_training` ou grafos separados para treinamento e teste.

### O Problema do Treinamento em Muitos Passos de Tempo

RNNs enfrentam o problema de **gradientes desvanecentes/explodindo** (discutido no Capítulo 11) quando treinadas em sequências muito longas. Isso dificulta a aprendizagem de dependências de longo prazo.

### Célula LSTM (Long Short-Term Memory)

A **célula LSTM** foi proposta em 1997 como uma solução para o problema dos gradientes desvanecentes. Ela consegue detectar dependências de longo prazo nos dados e converge mais rapidamente.
*   As células LSTM gerenciam **dois vetores de estado**: um **estado de curto prazo** `h(t)` e um **estado de longo prazo** `c(t)`.
*   Possui **três "portões" (gates)** que regulam o fluxo de informação:
    *   **Portão de Esquecimento (Forget Gate)**: Decide quais informações descartar do estado de longo prazo.
    *   **Portão de Entrada (Input Gate)**: Decide quais informações adicionar ao estado de longo prazo.
    *   **Portão de Saída (Output Gate)**: Decide qual parte do estado de longo prazo será lida e produzida como o estado de curto prazo e a saída.
*   **Conexões Peephole**: Uma melhoria que permite que os portões "observem" o estado de longo prazo. No TensorFlow, `tf.contrib.rnn.LSTMCell` com `use_peepholes=True` implementa isso.

### Célula GRU (Gated Recurrent Unit)

A **célula GRU** é uma versão simplificada da célula LSTM, introduzida em 2014. Ela também aborda o problema dos gradientes desvanecentes, mas tem uma arquitetura mais simples:
*   Combina os portões de esquecimento e entrada em um **portão de atualização**.
*   Não possui um portão de saída separado; o **vetor de estado completo é produzido em cada passo de tempo**.
*   Geralmente, as células GRU têm um desempenho semelhante às LSTMs.

### Processamento de Linguagem Natural (PNL)

As RNNs são fundamentais para PNL.

*   **Embeddings de Palavras (Word Embeddings)**:
    *   Representar palavras com vetores *one-hot* é ineficiente para grandes vocabulários, pois é esparso e não captura similaridades semânticas.
    *   A solução comum é representar cada palavra com um **vetor denso e de baixa dimensionalidade**, chamado **embedding**. Esses embeddings são aprendidos pela rede neural durante o treinamento, de modo que **palavras semanticamente similares tenham representações vetoriais próximas**.
    *   No TensorFlow, isso é implementado com `tf.Variable` para os embeddings e `tf.nn.embedding_lookup()` para obter o embedding de um ID de palavra.
*   **Rede Encoder-Decoder para Tradução Automática**:
    *   Um modelo comum consiste em um **encoder** (que processa a sentença de entrada) e um **decoder** (que gera a sentença traduzida).
    *   A sentença de entrada (por exemplo, inglês) é alimentada ao *encoder*, e as traduções (por exemplo, francês) são produzidas pelo *decoder*. As traduções francesas também são usadas como entradas para o *decoder*, deslocadas um passo de tempo para trás. Um token `<go>` é usado para a primeira palavra do decoder, e um token `<eos>` para indicar o fim da sentença de saída.
    *   É comum **inverter a sentença de entrada** para o *encoder* (`"Eu bebo leite"` torna-se `"leite bebo Eu"`), o que ajuda o *decoder* a ter acesso às primeiras palavras de entrada que são mais relevantes para o início da saída.
    *   No tempo de inferência (após o treinamento), o *decoder* usa sua **própria previsão da palavra anterior como entrada para o próximo passo de tempo**.
    *   **Detalhes Avançados**: Para otimizar o desempenho em tarefas de PNL com vocabulários grandes e comprimentos de frase variáveis, são utilizadas técnicas como:
        *   **Bucketing e Padding**: Agrupar sentenças por comprimentos semelhantes e preencher sentenças mais curtas com tokens `<pad>`.
        *   **Softmax Amostrado (Sampled Softmax)**: Uma técnica para estimar a perda em vocabulários muito grandes sem ter que calcular a função softmax sobre todas as palavras possíveis, o que seria computacionalmente intensivo.
        *   **Mecanismos de Atenção (Attention Mechanisms)**: Permitem que o *decoder* "olhe" para partes relevantes da sequência de entrada ao fazer uma previsão, melhorando significativamente a qualidade da tradução.

---

### Implementação

In [6]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds
import math

#### RNNs Básicas no TensorFlow

In [2]:
n_inputs = 3
n_neurons = 5
timesteps = 2
batch_size = 2

# Criando o modelo RNN
model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(n_neurons, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(timesteps, n_inputs))
])

# Dados de entrada
X = tf.random.normal((batch_size, timesteps, n_inputs))

# Forward pass
output = model(X)
print("Output shape:", output.shape)
print(output)

Output shape: (2, 2, 5)
tf.Tensor(
[[[-0.8424166   0.29035938 -0.2739727  -0.49007022 -0.5831856 ]
  [ 0.22340691 -0.20010251  0.6592095   0.6411391   0.55818844]]

 [[ 0.14533989  0.4443811   0.30226803 -0.41023073  0.0806545 ]
  [-0.9640687  -0.41499105 -0.70500284  0.775322   -0.50634515]]], shape=(2, 2, 5), dtype=float32)


c:\Users\andre\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [3]:
import tensorflow as tf
import numpy as np

# Dados de entrada combinados (batch_size, timesteps, features)
X_batch = np.array([
    [[0,1,2], [9,8,7]],    # amostra 1: t0 e t1
    [[3,4,5], [0,0,0]],    # amostra 2: t0 e t1  
    [[6,7,8], [6,5,4]],    # amostra 3: t0 e t1
    [[9,0,1], [3,2,1]]     # amostra 4: t0 e t1
], dtype=np.float32)

# Criar modelo RNN simples
model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(5, activation='tanh', return_sequences=True)
])

# Forward pass
output = model(X_batch)

print("Output shape:", output.shape)
print("Output at t=0:", output[:, 0, :].numpy())
print("Output at t=1:", output[:, 1, :].numpy())

Output shape: (4, 2, 5)
Output at t=0: [[ 0.65454847 -0.7930878  -0.79813707  0.8771043  -0.03404181]
 [ 0.5831427  -0.8046905  -0.9999951   0.9968557  -0.6669834 ]
 [ 0.5014251  -0.8157094  -1.          0.99992424 -0.91805726]
 [-0.84629315  0.99660844 -0.9999997  -0.94724274 -0.23252378]]
Output at t=1: [[-9.6196139e-01 -2.3785049e-01 -1.0000000e+00  9.9981207e-01
  -9.8382586e-01]
 [-6.7851985e-01 -8.9003277e-01  1.3682653e-01  6.7586333e-01
   1.6345747e-01]
 [-9.4484872e-01 -3.8059992e-01 -1.0000000e+00  9.9765360e-01
  -8.4935224e-01]
 [-7.0952058e-01  9.4043958e-01 -9.9921685e-01  8.5699535e-04
   7.3482496e-01]]


#### Desenrolamento estocástico através do tempo

In [4]:
import tensorflow as tf
import numpy as np

# Dados de entrada combinados
X_batch = np.array([
    [[0,1,2], [9,8,7]],  # Amostra 1: [t0, t1]
    [[3,4,5], [0,0,0]]   # Amostra 2: [t0, t1]
], dtype=np.float32)

# Criar modelo RNN
model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(n_neurons, 
                             activation='tanh',
                             return_sequences=True,
                             input_shape=(2, n_inputs))
])

# Obter saídas
output_sequence = model(X_batch)
y0 = output_sequence[:, 0, :]  # Primeiro timestep
y1 = output_sequence[:, 1, :]  # Segundo timestep

print("Shape da sequência de saída:", output_sequence.shape)
print("y0:", y0.numpy())
print("y1:", y1.numpy())

Shape da sequência de saída: (2, 2, 5)
y0: [[-0.9393268  -0.5258273  -0.8863454   0.6789909  -0.9827231 ]
 [-0.99975455  0.7592788  -0.99997675  0.71830785 -0.9999709 ]]
y1: [[-0.9999988   0.999221   -1.         -0.22900786 -0.9999992 ]
 [ 0.21299376 -0.7309075  -0.91290987  0.6290966   0.45839295]]


treinando um classificador de sequencia

In [5]:
n_steps = 28
n_inputs = 28
n_neurons = 150
n_outputs = 10

learning_rate = 0.001

model = tf.keras.Sequential([
	tf.keras.layers.SimpleRNN(n_neurons, activation='tanh', input_shape=(n_steps, n_inputs)),
	tf.keras.layers.Dense(n_outputs)
])

model.compile(
	optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
	loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
	metrics=['accuracy']
)

In [ ]:
import tensorflow as tf

# Carregar e pré-processar dados
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0
X_train = X_train.reshape(-1, 28, 28)
X_test = X_test.reshape(-1, 28, 28)

# Parâmetros
n_epochs = 100
batch_size = 150

# Modelo
model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(150, activation='tanh', input_shape=(28, 28)),
    tf.keras.layers.Dense(10)
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# Callbacks para monitoramento
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=3),
    tf.keras.callbacks.CSVLogger('training_log.csv')
]

# Treinar com fit() (maneira mais eficiente)
history = model.fit(
    X_train, y_train,
    batch_size=batch_size,
    epochs=n_epochs,
    validation_data=(X_test, y_test),
    callbacks=callbacks,
    verbose=1
)

# Avaliação final
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\nAcurácia final no teste: {test_acc:.4f}")

Epoch 1/100
190/400 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6911 - loss: 0.9835

### Treinando para prever séries temporais

In [ ]:
import tensorflow as tf

n_steps = 20
n_inputs = 1
n_neurons = 100
n_outputs = 1

# Usando Keras API (TF2 style)
model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(n_neurons, activation='relu', 
                             return_sequences=False,
                             input_shape=(n_steps, n_inputs)),
    tf.keras.layers.Dense(n_outputs)
])

# Alternativa: se você quer acesso às saídas de cada timestep
model_with_states = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(n_neurons, activation='relu',
                             return_sequences=True,  # Retorna todos os outputs
                             return_state=False,     # Não retorna estado final
                             input_shape=(n_steps, n_inputs)),
    tf.keras.layers.Dense(n_outputs)
])

# Compilar
model.compile(optimizer='adam', loss='mse')

# Exemplo de uso:
# X_batch shape: (batch_size, 20, 1)
# y_batch shape: (batch_size, 1)

c:\Users\andre\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
import tensorflow as tf
import numpy as np

# Parâmetros
n_steps = 20
n_inputs = 1
n_neurons = 100
n_outputs = 1
n_iterations = 1500
batch_size = 50

# Definir modelo
model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(n_neurons, activation='relu', 
                             input_shape=(n_steps, n_inputs)),
    tf.keras.layers.Dense(n_outputs)
])

# Compilar
model.compile(optimizer='adam', loss='mse')

# Função para gerar batches (substitua com seus dados)
def next_batch(batch_size, n_steps):
    # Dados de exemplo - substitua com dataset real
    X_batch = np.random.randn(batch_size, n_steps, n_inputs).astype(np.float32)
    y_batch = np.random.randn(batch_size, n_outputs).astype(np.float32)
    return X_batch, y_batch

# Dataset completo (exemplo)
X_full = np.random.randn(10000, n_steps, n_inputs).astype(np.float32)
y_full = np.random.randn(10000, n_outputs).astype(np.float32)

# Opção A: Usando fit() com dataset (Recomendado)
history = model.fit(
    X_full, y_full,
    batch_size=batch_size,
    epochs=n_iterations // (len(X_full) // batch_size),
    verbose=1
)

# Opção B: Treino manual batch a batch
print("Treinamento batch a batch:")
for iteration in range(n_iterations):
    X_batch, y_batch = next_batch(batch_size, n_steps)
    
    # Treinar no batch
    loss = model.train_on_batch(X_batch, y_batch)
    
    if iteration % 100 == 0:
        print(f"{iteration}\tMSE: {loss:.6f}")

# Avaliar
test_loss = model.evaluate(X_full[:1000], y_full[:1000], verbose=0)
print(f"\nMSE final no teste: {test_loss:.6f}")


Epoch 1/7


c:\Users\andre\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.9922
Epoch 2/7
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9845
Epoch 3/7
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9835
Epoch 4/7
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9807
Epoch 5/7
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9777
Epoch 6/7
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.9751
Epoch 7/7
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.9729
Treinamento batch a batch:
0	MSE: 0.972500
100	MSE: 0.986751
200	MSE: 1.009069
300	MSE: 1.008505
400	MSE: 1.011735
500	MSE: 1.005176
600	MSE: 1.006412
700	MSE: 1.006791
800	MSE: 1.005621
900	MSE: 1.006976
1000	MSE: 1.005481
1100	MSE: 1.003946
1200	MSE: 1.001801
1300	MSE: 1.001776
1400	MSE: 1.000297

MSE final no teste: 0.963784


### Exercícios

1. Você pode pensar em algumas aplicações para uma RNN de sequência-a-sequência? Que tal uma RNN sequência-vetor? E uma RNN vetor-a-sequência?

RNN Sequência-a-Sequência (Sequence-to-Sequence).
As RNNs sequência-a-sequência são amplamente utilizadas em tarefas onde tanto a entrada quanto a saída são sequências temporais. Aplicações notáveis incluem:

- **Tradução Automática**: Converter sequências de palavras de um idioma para outro (ex: Inglês → Português)
- **Sumarização de Texto**: Transformar documentos longos em resumos concisos
- **Tradução de Voz para Texto**: Converter sequências de áudio em texto transcrito
- **Chatbots e Sistemas de Diálogo**: Gerar respostas sequenciais baseadas em sequências de entrada
- **Análise de Sentimento em Tempo Real**: Processar fluxos contínuos de texto para análise de sentimentos
- **Correção Gramatical**: Identificar e corrigir erros em sequências textuais

RNN Sequência-a-Vetor (Sequence-to-Vector).
Esta arquitetura processa sequências e produz uma saída fixa (vetor). Aplicações comuns:

- **Classificação de Texto**: Analisar documentos inteiros para classificação por tópico ou sentimento
- **Análise de Séries Temporais**: Processar históricos temporais para prever categorias ou estados futuros
- **Reconhecimento de Fala**: Converter sequências de áudio em comandos ou palavras-chave específicas
- **Análise de Vídeo**: Processar frames sequenciais para classificação de ações ou atividades
- **Monitoramento de Equipamentos**: Analisar sequências de sensores para detectar falhas ou anomalias
- **Bioinformática**: Classificar sequências de DNA ou proteínas em categorias funcionais

RNN Vetor-a-Sequência (Vector-to-Sequence).
Esta arquitetura transforma uma entrada fixa (vetor) em uma sequência temporal. Aplicações interessantes:

- **Geração de Texto**: Criar textos, poemas ou histórias a partir de um vetor de seed ou prompt
- **Síntese de Voz**: Converter representações textuais ou semânticas em sequências de áudio
- **Geração de Música**: Compor sequências musicais a partir de representações de estilo ou emoção
- **Descrição de Imagens**: Gerar descrições textuais sequenciais a partir de representações vetoriais de imagens
- **Planejamento de Trajetórias**: Converter estados iniciais em sequências de movimentos ou ações
- **Sistemas de Recomendação Sequencial**: Gerar sequências de recomendações personalizadas baseadas em perfis de usuário

Cada arquitetura possui vantagens específicas dependendo da natureza do problema, sendo fundamental escolher a abordagem adequada para a tarefa em questão.

3. Como você poderia combinar uma rede neural convolucional com uma RNN para classificar vídeos?

Para classificar vídeos, uma abordagem comum é combinar redes neurais convolucionais (CNNs) com redes neurais recorrentes (RNNs), aproveitando o melhor de cada arquitetura:

- **CNNs** são excelentes para extrair características espaciais de imagens (por exemplo, identificar objetos, texturas e padrões em cada frame do vídeo).
- **RNNs** (como LSTM ou GRU) são ideais para modelar dependências temporais, ou seja, entender como as informações mudam ao longo do tempo entre os frames.

**Estratégia típica:**
1. **Extração de características por frame:**
   - Cada frame do vídeo é processado individualmente por uma CNN pré-treinada (como ResNet, VGG, etc.), gerando um vetor de características para cada frame.
2. **Sequência de vetores:**
   - Os vetores de características extraídos de todos os frames formam uma sequência temporal (um tensor de forma `[n_frames, n_features]`).
3. **Modelagem temporal:**
   - Essa sequência é então alimentada em uma RNN (LSTM, GRU, etc.), que aprende as relações temporais entre os frames.
4. **Classificação:**
   - A saída final da RNN (ou de uma camada densa subsequente) é usada para prever a classe do vídeo (por exemplo, ação, evento, categoria).

**Resumo visual:**
```
Frame 1   Frame 2   ...   Frame N
   |         |              |
 CNN       CNN            CNN
   |         |              |
Feature1  Feature2  ...  FeatureN
   |__________________________|
             |
           RNN
             |
        Classificação
```

**Vantagens:**
- Permite capturar tanto informações espaciais (o que está em cada frame) quanto temporais (como as coisas mudam ao longo do vídeo).
- Pode ser expandido para tarefas como detecção de ações, legendagem de vídeos, etc.

**Exemplo de uso em TensorFlow/Keras:**
- `TimeDistributed(CNN)` para aplicar a CNN em cada frame.
- `LSTM` ou `GRU` para processar a sequência de vetores de características.

Essa combinação é padrão em muitos sistemas de reconhecimento de vídeo modernos.

4. Quais são as vantagens de construir uma RNN utilizando dynamic_rnn() em vez de static_rnn()?

A. **Eficiência de Memória**:
    - `dynamic_rnn()` utiliza um loop dinâmico (`tf.while_loop`) para processar as sequências, o que resulta em um grafo computacional menor e mais eficiente em termos de memória.
    - Em contraste, `static_rnn()` desenrola a RNN completamente no tempo, criando uma cópia da célula para cada timestep, o que pode levar a problemas de falta de memória (OOM) para sequências longas.

B. **Flexibilidade com Comprimentos de Sequência Variáveis**:
    - `dynamic_rnn()` suporta diretamente sequências de comprimentos variáveis usando o parâmetro `sequence_length`. Isso permite que a RNN ignore os timesteps além do comprimento real da sequência, preenchendo-os com zeros automaticamente.
    - `static_rnn()` requer que todas as sequências tenham o mesmo comprimento, o que pode ser menos prático para dados reais.

C. **Desempenho Computacional**:
    - `dynamic_rnn()` é mais rápido, pois evita a criação de um grafo muito grande e utiliza operações otimizadas para loops dinâmicos.
    - `static_rnn()` pode ser mais lento devido ao tamanho do grafo e à sobrecarga de computação.

D. **Compatibilidade com TensorFlow Moderno**:
    - `dynamic_rnn()` é mais alinhado com as práticas modernas do TensorFlow, enquanto `static_rnn()` é considerado mais antigo e menos utilizado em implementações recentes.

E. **Facilidade de Debugging e Visualização**:
    - O grafo gerado por `dynamic_rnn()` é mais compacto e legível no TensorBoard, facilitando o debugging e a análise do modelo.


5. Como você pode lidar com sequencias de entrada de comprimento variavel? e quanto às sequencias de saída de tamanho variável?

- **Padding:** Preencha todas as sequências para o mesmo comprimento máximo do batch, adicionando um valor especial (geralmente zero) ao final das sequências mais curtas. Assim, todas as entradas têm o mesmo tamanho, facilitando o processamento em lote.
- **Máscara (Masking):** Utilize máscaras para informar à RNN quais timesteps são válidos e quais são apenas padding. Em Keras, pode-se usar a camada `Masking` ou passar o argumento `mask_zero=True` em camadas de embedding.
- **Parâmetro `sequence_length`:** Em APIs como `tf.nn.dynamic_rnn`, forneça um vetor com o comprimento real de cada sequência. A RNN ignora automaticamente os timesteps de padding durante o processamento e o cálculo do gradiente.

Para **sequências de saída de comprimento variável**:

- **Token de Fim de Sequência (EOS):** Adicione um token especial (por exemplo, `<eos>`) ao final de cada sequência de saída. Durante o treinamento, a função de custo só considera as saídas até o token `<eos>`, ignorando o restante.
- **Padding e Máscara na Saída:** Assim como na entrada, preencha as saídas para o mesmo comprimento e use máscaras para garantir que a função de custo só avalie os timesteps válidos.
- **Bucketing:** Agrupe sequências de tamanhos semelhantes em batches (buckets), reduzindo a quantidade de padding necessária e tornando o treinamento mais eficiente.

6. Qual é uma forma comum de se distribuir treinamento e execuçao de uma RNN profunda em várias GPUs?

Uma forma comum de distribuir o treinamento e a execução de uma RNN profunda em várias GPUs é dividir as camadas da RNN entre as GPUs disponíveis. Cada GPU processa uma parte do modelo, reduzindo a carga de memória e acelerando o treinamento. Aqui estão os passos principais:

A. **Divisão por Camadas**:
    - As camadas da RNN são distribuídas entre as GPUs. Por exemplo, se você tiver 4 GPUs e uma RNN com 4 camadas, cada GPU pode processar uma camada.

B. **Uso de `tf.distribute.MirroredStrategy`**:
    - O TensorFlow oferece a estratégia `tf.distribute.MirroredStrategy` para treinar modelos em várias GPUs. Ele replica o modelo em todas as GPUs e sincroniza os gradientes durante o treinamento.

C. **Definição de Dispositivos**:
    - Você pode usar `tf.device()` para especificar manualmente em qual GPU cada camada será executada. Por exemplo:
      ```python
      with tf.device('/gpu:0'):
            layer1 = tf.keras.layers.SimpleRNN(...)
      with tf.device('/gpu:1'):
            layer2 = tf.keras.layers.SimpleRNN(...)
      ```

D. **Divisão de Dados**:
    - Os dados de entrada são divididos em lotes menores, e cada GPU processa um lote. Isso pode ser feito automaticamente pelo TensorFlow ao usar `MirroredStrategy`.

E. **Sincronização de Gradientes**:
    - Após o cálculo dos gradientes em cada GPU, eles são sincronizados e combinados para atualizar os pesos do modelo.

**Exemplo com `MirroredStrategy`:**
```python
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
     model = tf.keras.Sequential([
          tf.keras.layers.SimpleRNN(100, return_sequences=True, input_shape=(n_steps, n_inputs)),
          tf.keras.layers.SimpleRNN(100),
          tf.keras.layers.Dense(1)
     ])
     model.compile(optimizer='adam', loss='mse')

model.fit(X_train, y_train, epochs=10, batch_size=64)

7. Embedded Reber grammars foram utilizadas por Hochreiter e Schmidhuber em seu artigo sobre LSTMs. São Gramáticas artificiais que produzem strings como "BPBTSXXVPSEPE". Confira a bela introdução de Jenny Orr sobre este tópico. Escolha uma gramática embutida específica (como a representada na página de Jenny Orr) depois treine uma RNN para identificar se uma string respeita ou não essa gramática. Você primeiro precisará escrever uma função capaz de gerar um lote de treinamento contendo cerca de 50% de strings que respeitem a gramática e 50% que não.

In [9]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.utils import to_categorical
import random

# Definição da gramática Embedded Reber
class EmbeddedReberGrammar:
    def __init__(self):
        self.vocab = {'B', 'T', 'V', 'X', 'P', 'S', 'E'}
        self.char_to_idx = {char: idx for idx, char in enumerate(sorted(self.vocab))}
        self.idx_to_char = {idx: char for char, idx in self.char_to_idx.items()}
        self.vocab_size = len(self.vocab)
    
    def generate_valid_string(self):
        """Gera uma string válida segundo a gramática Embedded Reber"""
        def generate_inner():
            # Primeiro caractere deve ser 'B'
            string = ['B']
            
            # Escolher entre caminho T ou V
            if random.random() < 0.5:
                string.append('T')
                # Desenvolver caminho T
                while random.random() < 0.7:  # Probabilidade de continuar
                    if random.random() < 0.5:
                        string.append('T')
                    else:
                        string.append('X')
                        if random.random() < 0.5:
                            string.append('S')
                        break
                else:
                    string.append('X')
            else:
                string.append('V')
                # Desenvolver caminho V
                while random.random() < 0.7:  # Probabilidade de continuar
                    if random.random() < 0.5:
                        string.append('V')
                    else:
                        string.append('P')
                        if random.random() < 0.5:
                            string.append('S')
                        break
                else:
                    string.append('P')
            
            # Finalizar com 'E'
            string.append('E')
            return ''.join(string)
        
        return generate_inner()
    
    def generate_invalid_string(self):
        """Gera uma string inválida introduzindo erros"""
        valid_string = self.generate_valid_string()
        string_list = list(valid_string)
        
        # Introduzir diferentes tipos de erros
        error_type = random.choice(['insert', 'delete', 'replace', 'swap'])
        
        if len(string_list) > 3:  # Garantir que há caracteres para modificar
            if error_type == 'insert' and len(string_list) < 15:
                # Inserir caractere aleatório
                pos = random.randint(1, len(string_list)-1)
                string_list.insert(pos, random.choice(list(self.vocab - {'B', 'E'})))
                
            elif error_type == 'delete' and len(string_list) > 4:
                # Deletar caractere (exceto B e E)
                pos = random.randint(1, len(string_list)-2)
                del string_list[pos]
                
            elif error_type == 'replace':
                # Substituir caractere
                pos = random.randint(1, len(string_list)-2)
                string_list[pos] = random.choice(list(self.vocab - {string_list[pos], 'B', 'E'}))
                
            elif error_type == 'swap' and len(string_list) > 4:
                # Trocar caracteres adjacentes
                pos = random.randint(1, len(string_list)-3)
                string_list[pos], string_list[pos+1] = string_list[pos+1], string_list[pos]
        
        return ''.join(string_list)
    
    def is_valid_string(self, string):
        """Verifica se uma string é válida segundo a gramática"""
        if not string.startswith('B') or not string.endswith('E'):
            return False
        
        # Remover B inicial e E final para análise
        core_string = string[1:-1]
        
        if not core_string:
            return False
        
        # Verificar padrões válidos
        valid_patterns = [
            'T' + 'X' * n + 'S' * m for n in range(1, 10) for m in range(2)
        ] + [
            'V' + 'P' * n + 'S' * m for n in range(1, 10) for m in range(2)
        ] + [
            'T' * n + 'X' + 'S' * m for n in range(1, 3) for m in range(2)
        ] + [
            'V' * n + 'P' + 'S' * m for n in range(1, 3) for m in range(2)
        ]
        
        return any(pattern in core_string for pattern in valid_patterns)
    
    def prepare_training_data(self, num_samples=10000, max_length=20):
        """Prepara dados de treinamento"""
        X = []
        y = []
        
        for _ in range(num_samples // 2):
            # Gerar string válida
            valid_str = self.generate_valid_string()
            X.append(valid_str)
            y.append(1)
            
            # Gerar string inválida
            invalid_str = self.generate_invalid_string()
            X.append(invalid_str)
            y.append(0)
        
        # Converter para sequences
        X_encoded = []
        for string in X:
            encoded = [self.char_to_idx[char] for char in string if char in self.char_to_idx]
            # Padding para tamanho máximo
            if len(encoded) < max_length:
                encoded += [0] * (max_length - len(encoded))
            else:
                encoded = encoded[:max_length]
            X_encoded.append(encoded)
        
        return np.array(X_encoded), np.array(y)

# Criar e preparar dados
grammar = EmbeddedReberGrammar()
X_train, y_train = grammar.prepare_training_data(10000)
X_test, y_test = grammar.prepare_training_data(2000)

print(f"Tamanho do vocabulário: {grammar.vocab_size}")
print(f"Shape dos dados de treinamento: {X_train.shape}")
print(f"Exemplo de string válida: {grammar.generate_valid_string()}")
print(f"Exemplo de string inválida: {grammar.generate_invalid_string()}")

# Construir modelo RNN
model = Sequential([
    Embedding(input_dim=grammar.vocab_size + 1, output_dim=16, input_length=20),
    LSTM(64, return_sequences=False),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

# Treinar o modelo
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_test, y_test),
    verbose=1
)

# Avaliar o modelo
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\nAcurácia no teste: {test_acc:.4f}")

# Função para testar strings personalizadas
def test_custom_strings(model, grammar, test_strings):
    """Testa strings personalizadas no modelo treinado"""
    results = []
    for string in test_strings:
        # Codificar string
        encoded = [grammar.char_to_idx.get(char, 0) for char in string]
        if len(encoded) < 20:
            encoded += [0] * (20 - len(encoded))
        else:
            encoded = encoded[:20]
        
        # Fazer predição
        prediction = model.predict(np.array([encoded]), verbose=0)[0][0]
        is_valid = prediction > 0.5
        actual_valid = grammar.is_valid_string(string)
        
        results.append({
            'string': string,
            'prediction': float(prediction),
            'predicted_valid': is_valid,
            'actual_valid': actual_valid,
            'correct': is_valid == actual_valid
        })
    
    return results

# Testar com exemplos
test_strings = [
    "BTXSE",      # Válido
    "BVPSE",      # Válido  
    "BTTXSE",     # Válido
    "BTTTSE",     # Inválido
    "BVPPE",      # Inválido
    "BXE",        # Inválido
    "BTPE",       # Inválido
    grammar.generate_valid_string(),
    grammar.generate_invalid_string()
]

results = test_custom_strings(model, grammar, test_strings)

print("\nResultados dos testes:")
for result in results:
    status = "✓" if result['correct'] else "✗"
    print(f"{status} '{result['string']}' -> Pred: {result['prediction']:.3f} "
          f"(Válido: {result['predicted_valid']}, Real: {result['actual_valid']})")

Tamanho do vocabulário: 7
Shape dos dados de treinamento: (10000, 20)
Exemplo de string válida: BTXE
Exemplo de string inválida: BVVVPE


c:\Users\andre\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.4988 - loss: 0.6934 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 2/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.4988 - loss: 0.6934 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 2/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.4950 - loss: 0.6933 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 3/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.4950 - loss: 0.6933 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 3/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.4953 - loss: 0.6933 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 4/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.4953 - loss: 0.6933 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 4/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5017 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 5/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5017 - loss: 0.6932 - val_accuracy: 0.

8. Encare a competição Kaggle "How much did it rain? II" (https://goo.gl/0DS5Xe). Esta é uma tarefa de previsão de séries temporais: São fornecidas fotografias de valores de radares polarimétricos e é solicitada a previsão do total de precipitação pluviométrica por hora. A entrevista de Luis André Dutra e Silva (htpps://goolgl/fTA90W) dá algumas informações interessantes sobre as técnicas que ele utilizou para alcançar o segundo lugar na competição. Em particular ele utilizou uma RNN composta de duas camadas LSTM.

9. Acesse o tutorial word2Vec (http://goo.gl/edArdi) do Tensorflow para criar uma word embedding e, em seguida, leia o tutorial Seq2Seq (https://goo.gl/L82gvs) para treinar um sistema de tradução do ingles para o francês.